In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json

# Define the path to your JSON file
file_path = '/content/drive/MyDrive/MasterThesis/captions_data_combined.json'

# Open and load the JSON data
with open(file_path, 'r') as file:
    captions_data = json.load(file)


In [ ]:
from transformers import pipeline

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")


def summarize_captions(generated_captions):
    combined_text = ' '.join(generated_captions)
    #Set max_length to be half of the input length, with a minimum value
    max_length = max(20, len(combined_text.split()) // 2)
    summarized_text = summarizer(combined_text, max_length=max_length, min_length=5, do_sample=False)[0]['summary_text']
    return summarized_text

model = summarizer.model

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

# Optionally, count only the trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Total Parameters: 305510400
Trainable Parameters: 305510400


In [ ]:
# Step 1: Install necessary libraries
!pip install transformers evaluate nltk

# Step 2: Import necessary libraries
import json
from transformers import pipeline
import evaluate
import nltk

# Load the BLEU metric from evaluate
bleu_metric = evaluate.load('bleu')


predictions = []
references = []


for item in captions_data:
    # Summarize generated captions
    generated_captions = list(item['generated_captions'].values())
    summarized_caption = summarize_captions(generated_captions)
    predictions.append(summarized_caption)

    # Original COCO captions as references
    original_captions = item['original_coco_captions']
    references.append(original_captions)  # Note the adjustment here

# Compute BLEU score with corrected structure
results = bleu_metric.compute(predictions=predictions, references=references)
print("BLEU Score:", results['bleu'])

BLEU Score: 0.27748611392761147


In [ ]:
print(predictions)

[' a person in a black hoodie a hot dog on a bun with mustard and k', ' a table with a plate of hot dogs and a cup of soda a hot dog and', ' a table with a plate of hot dogs and a cup of soda a hot dog and', ' a table with a plate of hot dogs and a cup of soda a hot dog and', ' a table with a plate of hot dogs and a cup of soda a hot dog and', ' a group of people standing around a motorcycle a person riding a motorcycle with a dog on', ' A group of people riding scoots on a street a number of people on a motorcycle', ' A dog laying in a dirty bathroom is a dog laying on the ground in a bathroom', ' A man is standing in front of a clock that says 11:00. A man', ' A man is standing in front of a clock that says 10:00. a bathroom', ' a dog standing in a bathroom with a toilet a dog is standing in the bathroom with', ' A man is holding a knife and a knife is next to a table . A man', ' a little girl sitting on the ground a young boy wearing a hat and a dog .', ' A bottle of wine is next to

In [ ]:
print(references)

[['Tray of food disguised in car, on restaurant counter', 'A plate that has a cardboard car holding a hot dog and fries in it.', 'some food is sticking out of a toy car', 'A green surface with  a fifties car model on a plate, topped with a hotdog roll and fries, a napkin dispenser, menus and condiments, has  two sets of  hands on either side, reaching for items, one of which is part of a black and  red/wearing woman. ', 'A meal of a hot dog and fries served out of a plate that looks like a muscle car.'], ['A plate with two hot dogs and a cup of coke on a table next to a bike.', 'Two hotdogs with mustard on a plate with a soft drink sitting on a table with a bike nearby.', 'Two hot dogs sit on a white paper plate near a soda cup which are sitting on a green picnic table while a bike and a silver car are parked nearby.', 'A picture of two hot dogs and a cup of soda', 'Table near car with a bicycle along side and a plate with two hot dogs and a coke.'], ['A plate with two hot dogs and a c

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Callidior/bert2bert-base-arxiv-titlegen")
model = AutoModelForSeq2SeqLM.from_pretrained("Callidior/bert2bert-base-arxiv-titlegen")


def generate_summary(input_text, tokenizer, model):
    # Tokenize input for the model
    inputs = tokenizer.encode("summarize: " + input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary with the model
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode and return the summary text
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

model = summarizer.model

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params}")

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Total Parameters: 247363386
Trainable Parameters: 247363386


In [ ]:
import evaluate

# Initialize BLEU metric
bleu_metric = evaluate.load('bleu')

predictions = []
references = []

for item in captions_data[:5]:  # Example: processing the first 5 items
    generated_captions = " ".join(list(item['generated_captions'].values()))
    summary = generate_summary(generated_captions, tokenizer, model)
    predictions.append(summary)
    references.append(item['original_coco_captions'])

# Compute BLEU score
results = bleu_metric.compute(predictions=predictions, references=[[ref] for ref in references])
print("BLEU Score:", results['bleu'])


BLEU Score: 0.0074574529265904925


In [ ]:
summarizer = pipeline("summarization", model="Einmalumdiewelt/T5-Base_GNAD")

model = summarizer.model

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params}")

Total Parameters: 222882048
Trainable Parameters: 222882048


In [ ]:
# Load the BLEU metric from evaluate
bleu_metric = evaluate.load('bleu')

predictions = []
references = []

for item in captions_data:
    # Summarize generated captions
    generated_captions = list(item['generated_captions'].values())
    summarized_caption = summarize_captions(generated_captions,summarizer)
    print(summarized_caption)
    predictions.append(summarized_caption)

    # Original COCO captions as references
    original_captions = item['original_coco_captions']
    references.append(original_captions)  # Note the adjustment here

# Compute BLEU score with corrected structure
results = bleu_metric.compute(predictions=predictions, references=references)
print("BLEU Score:", results['bleu'])


A man in a black hoodie eats a hot dog on 
A hot dog and a drink on a picnic table A bottle of Coke sits
A hot dog and a drink on a picnic table A bottle of Coke sits
A hot dog and a drink on a picnic table A bottle of Coke sits
A hot dog and a drink on a picnic table A bottle of Coke sits
A group of people are standing in front of a building that says "The Museum".
A group of people riding scoots on a street A man riding a
A man is holding a bottle of wine on it and holding it in his hand: 
A man riding a bike on a city street, next to a park, in
In a bathroom with marble walls and a large mirror, a man is standing in
In a bathroom with a toilet a man is holding a box of alcohol.
A man is holding a knife and a dog is laying on the ground next to
A little girl sitting on the ground, a boy wearing a hat and a
In a kitchen, a bottle of wine is next to a glass of wine.
"Don't Feed Da Bears" is a book written by a man
A dog with a scarf on its head A man holds a book titled Canada.
In a bo

In [ ]:
summarizer = pipeline("summarization", model="google/bigbird-pegasus-large-pubmed")

model = summarizer.model

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params}")

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.31G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.92M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

Total Parameters: 576891904
Trainable Parameters: 576891904


In [ ]:
# Load the BLEU metric from evaluate
bleu_metric = evaluate.load('bleu')


predictions = []
references = []


for item in captions_data:
    # Summarize generated captions
    generated_captions = list(item['generated_captions'].values())
    summarized_caption = summarize_captions(generated_captions,summarizer)
    print(summarized_caption)
    predictions.append(summarized_caption)

    # Original COCO captions as references
    original_captions = item['original_coco_captions']
    references.append(original_captions)  # Note the adjustment here

# Compute BLEU score with corrected structure
results = bleu_metric.compute(predictions=predictions, references=references)
print("BLEU Score:", results['bleu'])


Attention type 'block_sparse' is not possible if sequence_length: 29 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


this book is a combination of a book and a demonstration .<n> it is a combination of
a small herd of sphynx breed dogs was presented at the 35th annual federation of
a small herd of sphynx breed dogs was presented at the 35th annual federation of
a small herd of sphynx breed dogs was presented at the 35th annual federation of
a small herd of sphynx breed dogs was presented at the 35th annual federation of
we present a patient with end - stage renal disease ( esrd ) in whom a
 a group of people riding their motorcycle on a street with a sign that 16 says 
key clinical messagea dog laying in a dirty bathroom is holding a bottle of wine .<n>
the world health organization ( who ) released a report on two years ago what seemed to be
key clinical messagea man with a cerebral hemorrhage found in a bathroom with marble walls and a large
key clinical messagea dog standing in a bathroom with a toilet with a toilet is standing in the
key clinical messagea dog laying on the ground next to a bike a

In [ ]:
summarizer = pipeline("summarization", model="VietAI/vit5-large-vietnews-summarization")

model = summarizer.model

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

# Optionally, count only the trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params}")

Total Parameters: 791276544
Trainable Parameters: 791276544


In [ ]:
# Load the BLEU metric from evaluate
bleu_metric = evaluate.load('bleu')


predictions = []
references = []


for item in captions_data:
    # Summarize generated captions
    generated_captions = list(item['generated_captions'].values())
    summarized_caption = summarize_captions(generated_captions,summarizer)
    print(summarized_caption)
    predictions.append(summarized_caption)

    # Original COCO captions as references
    original_captions = item['original_coco_captions']
    references.append(original_captions)  # Note the adjustment here


results = bleu_metric.compute(predictions=predictions, references=references)
print("BLEU Score:", results['bleu'])


a person in a black hoodie a hot dog on a bun with mustard và
a table with a plate of hot dogs and a cup of soda a hot dog
a table with a plate of hot dogs and a cup of soda a hot dog
a table with a plate of hot dogs and a cup of soda a hot dog
a table with a plate of hot dogs and a cup of soda a hot dog
Một nhóm người ở "The Museum " , chụp hình cùng a motorcycle in front
" The motorcycle of people riding scoots on a street "
A dog laying in a dirty bathroom a dog lay động in the ground
a man riding a bike on a city street a man estate a b
Trong một nhà kho cũ ở Berlin , người ta thấy một con kangaroo đang nằm trên mặt đất
a dog is standing in the bathroom with a toilet A man is holding a
a dog laying on the blood in the sidewalk next to a b
Một số thành phố ở Mỹ như New York , London , Rome , San Francisco , Stockholm ,
a black dog laying on the floor in a kitchen a black girl laying
a woman standing in a kitchen with a dog "Don't Fe
a dog wearing a shirt a dog with a scf on its head

In [ ]:
summarizer = pipeline("summarization", model="kabita-choudhary/finetuned-bart-for-conversation-summary")

model = summarizer.model

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_params}")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params}")

In [ ]:
# Load the BLEU metric from evaluate
bleu_metric = evaluate.load('bleu')


predictions = []
references = []


for item in captions_data:
    # Summarize generated captions
    generated_captions = list(item['generated_captions'].values())
    summarized_caption = summarize_captions(generated_captions,summarizer)
    print(summarized_caption)
    predictions.append(summarized_caption)

    # Original COCO captions as references
    original_captions = item['original_coco_captions']
    references.append(original_captions)  # Note the adjustment here

# Compute BLEU score with corrected structure
results = bleu_metric.compute(predictions=predictions, references=references)
print("BLEU Score:", results['bleu'])
